<a href="https://colab.research.google.com/github/laruvinga/machinelearning/blob/master/Classify_Informal_Settlements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import all relevant libraries for model that will be using Keras.**

In [0]:
import keras
import h5py
import PIL.Image
import os, shutil
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.inception_v3 import preprocess_input
import os
import pprint
import tensorflow as tf
import shutil
from sklearn.metrics import classification_report, confusion_matrix

In [137]:
print(tf.__version__)
print(tf.keras.__version__)

1.14.0
2.2.4-tf


In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset_dir = "/content/drive/My Drive/SatelliteML/Test_101"
dataset = "/tmp/byo_ngozi.tar.bz2"
train_dir = "/tmp/dataset/train";
valid_dir = "/tmp/dataset/valid";
dataset_url = "https://github.com/laruvinga/machinelearning/raw/master/byo_ngozi.tar.bz2"

In [140]:
tf.keras.utils.get_file(dataset, dataset_url)

'/tmp/byo_ngozi.tar.bz2'

In [0]:
shutil.unpack_archive(dataset, "/tmp/dataset", "bztar")

In [142]:
#train using data augmentation and dropout
train_datagen = ImageDataGenerator(
                    preprocessing_function=preprocess_input,
                    rescale = 1./255,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255) #validation data should not be augmented

train_generator = train_datagen.flow_from_directory(
                    train_dir, 
                    target_size = (28,28),
                    batch_size = 1,
                    class_mode = 'binary')
validation_generator = validation_datagen.flow_from_directory(
                  valid_dir, 
                  target_size = (28,28),
                  batch_size = 1,
                  class_mode = 'binary')

STEP_SIZE_TRAIN = 16 # train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = 10 # validation_generator.n//validation_generator.batch_size

Found 58 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [0]:
def create_model():
  # with strategy.scope():
  model = models.Sequential()
  model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (150,150,3)))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(256,(3,3), activation = 'relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation = 'relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation = 'sigmoid'))
  return model

In [144]:
model = create_model()
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# model = create_model2()
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 64)              

In [145]:

#train the model with fit_generator
history = model.fit_generator(
      train_generator,
      steps_per_epoch= STEP_SIZE_TRAIN,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=STEP_SIZE_VALID
 )

# history = model.fit(train_ds, epochs=10, steps_per_epoch=100, shuffle=False, workers=8)

#Save model after training
model.save(dataset_dir + '/slums_model_100.h5', overwrite=True)

Epoch 1/100
16/16 [==============================] - 2s 104ms/step - loss: 0.6916 - acc: 0.6250 - val_loss: 0.7101 - val_acc: 0.5000
Epoch 2/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6998 - acc: 0.3750 - val_loss: 0.7012 - val_acc: 0.5000
Epoch 3/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6938 - acc: 0.5000 - val_loss: 0.6967 - val_acc: 0.5000
Epoch 4/100
16/16 [==============================] - 0s 21ms/step - loss: 0.6929 - acc: 0.5625 - val_loss: 0.6959 - val_acc: 0.5000
Epoch 5/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6957 - acc: 0.2500 - val_loss: 0.6953 - val_acc: 0.5000
Epoch 6/100
16/16 [==============================] - 0s 17ms/step - loss: 0.6935 - acc: 0.4375 - val_loss: 0.6956 - val_acc: 0.5000
Epoch 7/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6941 - acc: 0.5000 - val_loss: 0.6950 - val_acc: 0.5000
Epoch 8/100
16/16 [==============================] - 0s 22ms/step - loss: 0